## 랭체인의 PDF를 읽는 다양한 방법
랭체인에서는 PDF 파일을 읽고 처리할 수 있는 다양한 PDF Loader 를 제공합니다. 다음 주소에서 랭체인에서 제공하는 모든 PDF Loader 들을 확인할 수 있습니다. 이번 절에서는 그중 가장 많이 쓰는 몇 가지 PDF Loader 만 소개합니다. 그중 여기서 배운 PyPDFLodaer 를 활용하여 뒤에서 RAG 챗봇을 제작하게 됩니다.

• 링크: https://python.langchain.com/v0.1/docs/modules/data_connection/document_loaders/pdf/

In [1]:
!pip install langchain_community pypdf pymupdf pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.2/313.2 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 77.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 95.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 76.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.8 MB/s eta 0:00:00


###1. PyPDFLoader
PyPDFLoader는 가장 많이 쓰이는 PDF Loader 로서 일반적으로 페이지 단위로 로드합니다. 예를 들어,
총 27 페이지로 구성된 PDF 라면 27 개의 청크로 로드합니다. 다만 이것이 항상 보장되지는 않는데, 이는 특정 페이지에 텍스트가 없이 그림만 있거나 텍스트가 없는 백지 페이지의 경우 해당 페이지는 건너뛰므로 로드된 청크가 기존 PDF 페이지의 수보다 적은 경우도 있습니다. 여기서는 통일부에서 발간한 ‘2023_북한인권보고서.pdf’로 실습을 진행해보겠습니다.

In [3]:
import urllib.request
from langchain.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.document_loaders import PDFPlumberLoader

2023_ 북한인권보고서.pdf 파일을 온라인 코드 저장소에서 다운로드합니다.

In [5]:
urllib.request.urlretrieve(
    "https://github.com/chatgpt-kr/openai-api-tutorial/raw/main/ch06/2023_%EB%B6%81%ED%95%9C%EC%9D%B8%EA%B6%8C%EB%B3%B4%EA%B3%A0%EC%84%9C.pdf",
    filename="2023_북한인권보고서.pdf")

('2023_북한인권보고서.pdf', <http.client.HTTPMessage at 0x7f5248216190>)

이번 실습에서는 여러 PDF Loader 의 처리 속도를 비교해보고자 합니다. 주피터 노트북 실행 시 %%time
을 사용하면 코드 처리 속도를 측정할 수 있습니다. 이제 랭체인의 PyPDFLoader()를 통해 PDF 파일
을 로드합니다. PyPDFLoader(파일명)을 실행하여 loader라는 객체를 선언하고, 해당 객체를 통
해 load_and_split()을 실행하면 PDF 를 여러 개의 문서 청크로 분할한 문자열 리스트가 반환됩니
다.

In [7]:
%%time
loader = PyPDFLoader('2023_북한인권보고서.pdf')
pages = loader.load_and_split()
print('청크의 수:', len(pages))

청크의 수: 445
CPU times: user 15 s, sys: 91.4 ms, total: 15.1 s
Wall time: 15.2 s


청크의 수는 445 가 나왔습니다. 실제 이 PDF 의 페이지 수는 448 페이지이지만, 텍스트가 없고 그림만 있는 페이지나 백지 등의 페이지는 제외하고, 각 페이지가 청크로 할당되어 445 개의 청크로 분할되었습니다. 아래에는 CPU times 와 Wall time 이 나와있고, 또 CPU times 는 user 시간과 sys 시간으로 나뉩니다.
이를 각각 사용자 시간과 시스템 시간이라고 합니다. 이를 정리하면 다음과 같습니다.

- 사용자 시간 (User Time): 15.2 초 동안 컴퓨터는 여러분이 작성한 프로그램 코드를 처리하는 데 집
중했습니다. 이 시간은 실제로 여러분의 코드가 계산하거나 작업하는 데 사용된 시간입니다.

- 시스템 시간 (System Time): ms 는 밀리초라는 의미입니다. 55.9 밀리초는 컴퓨터가 파일을 열거
나 네트워크와 같은 다른 작업을 하는 데 사용한 시간입니다. 밀리초는 1 초의 천분의 일입니다.

- 벽시계 시간 (Wall Time): 15.5 초는 프로그램을 시작하고 끝날 때까지 실제로 걸린 시간입니다. 이
시간은 여러분이 실제로 기다린 전체 시간을 말해 줍니다. 이렇게 보면, 컴퓨터는 대부분의 시간을
프로그램 코드를 실행하는 데 사용했고, 전체적으로는 15.5 초가 걸렸다고 이해할 수 있습니다. 정
확하게 잘 분리되었는지 확인하기 위해 임의로 3 번 청크를 출력해보겠습니다.

In [8]:
pages[3]

Document(metadata={'producer': 'Adobe PDF Library 10.0.1', 'creator': 'Adobe InDesign CS6 (Windows)', 'creationdate': '2023-07-31T13:50:27+09:00', 'moddate': '2023-07-31T13:57:54+09:00', 'trapped': '/False', 'source': '2023_북한인권보고서.pdf', 'total_pages': 448, 'page': 5, 'page_label': '6'}, page_content='2023 북한인권보고서\n04\n올해로 유엔의 북한인권조사위원회 출범 10년, 북한인권결의 채\n택 20년이 됩니다. 그동안 우리는 물론 국제사회가 북한인권을 증진\n하기 위해 노력해 왔지만, 휴전선 이북의 북녘 땅은 여전히 최악의 \n인권 사각지대로 남아 있습니다. 우리와 피를 나눈 북한 동포들이 \n최소한의 인간적인 삶을 누릴 수 있도록 책임감을 갖고 보다 실효적\n인 노력을 펼쳐가야만 합니다. \n2016년 제정된 북한인권법에 기반하여 설립된 북한인권기록센\n터는 2017년부터 북한이탈주민을 대상으로 북한의 전반적인 인권\n실태를 심층적으로 조사하였습니다. 또한 파악된 북한의 인권침해 \n사례들을 ‘세계인권선언’과 ‘국제인권조약’의 기준에 따라 분류하였\n습니다. 이번에 발간되는 「북한인권보고서」는 북한의 인권 상황을 \n시민적·정치적 권리, 경제적·사회적·문화적 권리 등 다양한 측면에\n서 입체적으로 조명하였습니다. 아울러, 여성·아동·장애인 등 취약\n계층, 정치범수용소 및 국군포로·납북자·이산가족 등 특수 인권문\n발간사')

보다시피 page_content에 분할된 텍스트의 본문이 저장되어 있고, source에는 해당 본문의 원
본 파일의 이름이 저장되어 있습니다. 기본적으로 랭체인에서 PyPDFLoader()로 객체를 선언하고
load_and_split()을 사용하는 경우에는 다음과 같은 형식을 따릅니다.

Document(page_content = '내용', metadata = {'source': 파일명, 'page': 기존 PDF파일에서의 페이지 번호})

RecursiveCharacterTextSplitter에서 소
개한 실습과 같이 각 청크에 .page_content를 붙여서 호출하면 됩니다. 이제 다른 PDF Loader 들을
살펴봅시다.

###2. PyPDFLoader
PyMuPDFLoader는 속도가 다른 PDF 로더보다 빠르며, 메타데이터가 PyPDFLoader보다 훨씬 다양
하다는 특징을 갖고 있습니다. 속도가 빠르므로 PDF 파일이 많거나 긴 경우에 사용하는 것을 권장합니다.
또한 청크 결과가 다른 PDF Loader 와 다른 경우도 있으니 결과를 비교하여 선택하는 것도 필요할 수 있습니다. 이제 랭체인의 PyMuPDFLoader()를 통해 PDF 파일을 로드합니다. PyMuPDFLoader(파일명
)을 실행하여 loader라는 객체를 선언하고, 해당 객체를 통해 load_and_split()을 실행하면 PDF
를 여러 개의 문서 청크로 분할한 문자열 리스트가 반환됩니다.

In [10]:
%%time
loader = PyMuPDFLoader('2023_북한인권보고서.pdf')
pages = loader.load_and_split()
print('청크의 수: ', len(pages))

청크의 수:  445
CPU times: user 2.33 s, sys: 20.3 ms, total: 2.35 s
Wall time: 2.35 s


실행 결과를 보면 청크의 수는PyPDFLoader와 동일하게 445 개이며, 실행 시간은 벽시계 시간 기준으로 2.35 초로, PyPDFLoader와 비교하여 더욱 짧은 시간이 걸렸음을 알 수 있습니다. 즉, 동일한 문서에 대해 PyMuPDFLoader가 PyPDFLoader보다 더 빠른 처리 시간을 보여줍니다. 정확하게 잘 분리되었는지 확인하기 위해 임의로 3번 청크를 출력하겠습니다.

In [11]:
pages[3]

Document(metadata={'producer': 'Adobe PDF Library 10.0.1', 'creator': 'Adobe InDesign CS6 (Windows)', 'creationdate': '2023-07-31T13:50:27+09:00', 'source': '2023_북한인권보고서.pdf', 'file_path': '2023_북한인권보고서.pdf', 'total_pages': 448, 'format': 'PDF 1.6', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2023-07-31T13:57:54+09:00', 'trapped': '', 'modDate': "D:20230731135754+09'00'", 'creationDate': "D:20230731135027+09'00'", 'page': 5}, page_content='2023 북한인권보고서\n04\n올해로 유엔의 북한인권조사위원회 출범 10년, 북한인권결의 채\n택 20년이 됩니다. 그동안 우리는 물론 국제사회가 북한인권을 증진\n하기 위해 노력해 왔지만, 휴전선 이북의 북녘 땅은 여전히 최악의 \n인권 사각지대로 남아 있습니다. 우리와 피를 나눈 북한 동포들이 \n최소한의 인간적인 삶을 누릴 수 있도록 책임감을 갖고 보다 실효적\n인 노력을 펼쳐가야만 합니다. \n2016년 제정된 북한인권법에 기반하여 설립된 북한인권기록센\n터는 2017년부터 북한이탈주민을 대상으로 북한의 전반적인 인권\n실태를 심층적으로 조사하였습니다. 또한 파악된 북한의 인권침해 \n사례들을 ‘세계인권선언’과 ‘국제인권조약’의 기준에 따라 분류하였\n습니다. 이번에 발간되는 「북한인권보고서」는 북한의 인권 상황을 \n시민적·정치적 권리, 경제적·사회적·문화적 권리 등 다양한 측면에\n서 입체적으로 조명하였습니다. 아울러, 여성·아동·장애인 등 취약\n계층, 정치범수용소 및 국군포로·납북자·이산가족 등 특수 인권문\n발간사'

PyPDFLoader로 수행했을 때와 page_content의 값이 줄바꿈 등에서 다소 차이를 보입니다. 또한
metadata에 훨씬 다양한 정보가 들어 있다는 점이 큰 차이점입니다.

###3. PDFPlumberLoader
PDFPlumberLoader는 PyMuPDFLoader와 마찬가지로 metadata가 PyPDFLoader보다 훨씬 다양하다는 특징이 있습니다. 단, 읽기 속도가 상대적으로 느립니다. 이제 랭체인의 PDFPlumberLoader() 통해 PDF 파일을 로드합니다. PDFPlumberLoader(파일명)을 실행하여 loader 라는 객체를 선언하고, 해당 객체를 통해 load_and_split()을 실행하면 PDF 를 여러 개의 문서 청크로 분할한 문자열 리스트가 반환됩니다.

In [12]:
%%time
loader = PDFPlumberLoader('2023_북한인권보고서.pdf')
pages = loader.load_and_split()
print('청크의 수: ', len(pages))

청크의 수:  445
CPU times: user 25.8 s, sys: 1.81 s, total: 27.6 s
Wall time: 27.8 s


실행결과를 보면 청크의수는 PyPDFLoader와 동일하게 445개이며, 실행시간은 벽시계 시간기준으
로 27.8초로 가장 긴 시간이 걸렸음을 알수있습니다.정확하게  잘 분리 되었는지 확인하기 위해 임의로 3번 청크를 출력해 보겠습니다.

In [13]:
pages[3]

Document(metadata={'source': '2023_북한인권보고서.pdf', 'file_path': '2023_북한인권보고서.pdf', 'page': 5, 'total_pages': 448, 'CreationDate': "D:20230731135027+09'00'", 'Creator': 'Adobe InDesign CS6 (Windows)', 'ModDate': "D:20230731135754+09'00'", 'Producer': 'Adobe PDF Library 10.0.1', 'Trapped': 'False'}, page_content='발간사\n올해로 유엔의 북한인권조사위원회 출범 10년, 북한인권결의 채\n택 20년이 됩니다. 그동안 우리는 물론 국제사회가 북한인권을 증진\n하기 위해 노력해 왔지만, 휴전선 이북의 북녘 땅은 여전히 최악의\n인권 사각지대로 남아 있습니다. 우리와 피를 나눈 북한 동포들이\n최소한의 인간적인 삶을 누릴 수 있도록 책임감을 갖고 보다 실효적\n인 노력을 펼쳐가야만 합니다.\n2016년 제정된 북한인권법에 기반하여 설립된 북한인권기록센\n터는 2017년부터 북한이탈주민을 대상으로 북한의 전반적인 인권\n실태를 심층적으로 조사하였습니다. 또한 파악된 북한의 인권침해\n사례들을 ‘세계인권선언’과 ‘국제인권조약’의 기준에 따라 분류하였\n습니다. 이번에 발간되는 「북한인권보고서」는 북한의 인권 상황을\n시민적·정치적 권리, 경제적·사회적·문화적 권리 등 다양한 측면에\n서 입체적으로 조명하였습니다. 아울러, 여성·아동·장애인 등 취약\n계층, 정치범수용소 및 국군포로·납북자·이산가족 등 특수 인권문\n04 2023 북한인권보고서')

 PyPDFLoader로 수행 했을때와 page_content의 값이 줄바꿈 등에서 다소 차이를 보입니다.또한
metadata에 훨씬 다양한 정보가 들어 있다는 점이 차이점입니다.


